# TME 4 - Attaque par Analyse de Consommation Differentielle (DPA)

## Théorie de l'attaque DPA

Comme nous l'avons vu précédemment, le poids de Hamming du résultat de l'opération SBox dans l'AES a un effet mesurable sur la puissance consommée par le microcontrôleur. Il s'avère que cet effet (et non quelque chose de plus fort, comme sa linéarité) est une information suffisante pour casser une clé AES. Il y a plusieurs façons de procéder, mais pour ce TME, nous nous intéresserons à la différence des moyennes. Avec cette technique, l'objectif est de séparer les traces en 2 groupes selon la valeur d'un ou plusieurs bit(s) du résultat de la sortie de la SBox.

Si l'on considère un bit de la sortie de la SBox, le groupe de traces pour lequel sa valeur est de 1 devrait, en moyenne, avoir une consommation d'énergie plus élevée pendant l'opération SBox que l'autre ensemble pour lequel sa valeur est de 0.

Pour la DPA multibits, il faut trouver un moyen de combiner les valeurs des différents bits pour former deux groupes pertinents : la fonction HW() de la sortie Sbox peut être utilisée pour diviser les traces en deux groupes, celles dont les valeurs sont supérieures à 4 et celles dont les valeurs sont inférieures à 4. Les traces avec un HW de 4 peuvent être exclues ou ajoutées à un groupe.

L'existence ou non d'une différence importante dans les moyennes entre ces deux groupes dépend du fait qu'ils ont été correctement triés dans ces groupes. Si ce n'est pas le cas, il devrait y avoir, en moyenne, peu de différence entre les deux et donc une faible différence de moyennes. Rappelons l'opération SBox :

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

La sortie de la SBox dépend de la sous-clé, que nous ne connaissons pas (et du texte en clair, que nous connaissons). Cependant, étant donné qu'il existe une grande différence de moyennes pour la clé correcte et de petites différences pour les autres sous-clés possibles, nous disposons d'une méthode pour vérifier si une sous-clé donnée est correcte. Si nous calculons la différence de moyennes pour chaque sous-clé, la bonne clé aura la plus grande différence de moyennes.

L'attaque se compose donc des étapes suivantes (en considérant la DPA sur un bit d'un octet) :
1. Capturer les traces de puissance avec un texte en clair variable (et une clé fixe !)
1. Classifier chaque trace en fonction de la valeur de la sortie de la SBox pour une supposition de sous-clé donnée.
1. Calculer la différence des moyennes (il s'agit d'une trace)
1. Répéter l'opération pour chaque sous-clé possible
1. Sélectionner la plus grande différence de moyennes -> il devrait s'agir de la bonne sous-clé
1. Répéter l'opération pour chaque sous-clé de la clé

À la fin, nous devrions obtenir une clé AES correcte !

## Réalisation de l'attaque DPA en considérant un bit

In [ ]:
from tqdm.notebook import tnrange
import numpy as np
import time


SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'
%run "Setup_Scripts/Setup_Generic.ipynb"
fw_path = "../hardware/victims/firmware/simpleserial-aes-sca/simpleserial-aes-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

scope.adc.samples = 1000       # to be adapted if needed (10000 is the default) 870 
scope.adc.offset = 1557       # to be adapted if needed (0 is the default) 1550
period = 48
sb_samples = 38
subloop_samples = 10
mainloop_samples = 28

offset = scope.adc.offset
samples = 16*period + 4*mainloop_samples

def byte_range(b) :
    q = b // 4
    r = b % 4
    s = (b*period + q*mainloop_samples)
    return s, s+sb_samples 

# Capture Traces

ktp = cw.ktp.Basic() # object dedicated to the generation of key (fixed by default) and plain text
#key, text = ktp.next()
N = 1024  # Number of traces
nkeys = 16

for j in range(nkeys) :
    traces = [] # list of traces
    ktp.fixed_key = False
    key, text = ktp.next()
    ktp.fixed_key = True
    #print(key)
    for i in tnrange(N, desc = 'Capturing traces'):
        key, text = ktp.next()  # creation of a pair comprising (fixed) key and text 
        #print(key)
        trace = cw.capture_trace(scope, target, text, key) # a trace is composed of the following fields :
                                                           #    a wave (samples)
                                                           #    textin (input text), textout (output text)
                                                           #    key (input key)
        if trace is None:
            continue
        traces.append(trace)  

    # Convert traces to numpy arrays 
    trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
    textin_array = np.asarray([trace.textin for trace in traces])
    known_keys = np.asarray([trace.key for trace in traces])    # for fixed key generation, these keys are all the same

    print(known_keys)
    np.save("k"+str(j)+"_textin_array.npy", textin_array)
    np.save("k"+str(j)+"_known_keys.npy", known_keys)
    np.save("k"+str(j)+"_trace_array.npy", trace_array)

In [ ]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]


In [ ]:
def load_traces() :
    trace_array = np.load("trace_array.npy")
    textin_array = np.load("textin_array.npy")
    known_keys = np.load("known_keys.npy")

    
def load_traces_nkeys(key_number) :
    trace_array = []
    textin_array = []
    known_keys = []
    a = np.load("k"+str(key_number)+"_trace_array.npy")
    trace_array = a
    b = np.load("k"+str(key_number)+"_textin_array.npy")
    textin_array = b
    c = np.load("k"+str(key_number)+"_known_keys.npy")
    known_keys = c
    print(known_keys[0])
        
def compare_keys(actual_key, found_key) :
    res = []
    for i in range(16) :
        res.append(actual_key[2*i:2*i+2] == found_key[2*i:2*i+2])
    return res

def error_count(actual_key, found_key) :
    return compare_keys(actual_key, found_key).count(False)



In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
from random import randrange

alpha = 2

hw_leak = [49, 269, 488, 708, 96, 316, 536, 756, 144, 364, 584, 804, 220, 440, 660, 836]
    
def attack_byte_sbit(bnum) :
    # best key byte guesses for each bit
    guesses = []
    # maximum difference for each bit
    diff_arr = []
    for i in range(0, 8) :
        M = 0
        best_key = 0
        for key_guess in range(256) :
            b_0 = []
            b_1 = []
            for tnum in range(0, N): 
                textin = textin_array[tnum][bnum]
                sbox_out = intermediate(textin, key_guess)
                b = (sbox_out) & (0x1 << i)
                # sort the traces depending on the sbox output
                if b == 0 :
                    b_0.append(tnum)
                else : 
                    b_1.append(tnum)
            # compute the range depending on the critical point we found
            ra = range(hw_leak[bnum] - alpha, hw_leak[bnum] + alpha + 1)
            
            mean_trace_0 = [0] * len(ra)
            mean_trace_1 = [0] * len(ra)
            # compute the mean traces for each group
            for j in range(len(ra)) :
                mean_trace_0[j] = np.mean([trace_array[tnum][ra[0]+j] for tnum in b_0])
                mean_trace_1[j] = np.mean([trace_array[tnum][ra[0]+j] for tnum in b_1])
            # compute the difference between the two mean traces
            diff = np.array(mean_trace_0) - np.array(mean_trace_1)
            sabs = np.sum(np.abs(diff))

            # store the byte that has the largest mean value difference s
            if sabs > M :
                M = sabs
                best_key = key_guess
        # append the best byte to the guess array 
        diff_arr.append(M)
        # the maximum difference
        guesses.append(best_key)
    return guesses[diff_arr.index(max(diff_arr))]

    
def attack_key_sbit() :
    key = ""
    #print(known_keys[0])
    for i in range(0, 16) :
        by = known_keys[0][i]
        key += "{:02x}".format(by)
    password = ""
    for bnum in range(16) :
        by = attack_byte_sbit(bnum)
        password += "{:02x}".format(by)
    #print("found password = \t", password)
    #print("actual password = \t", key)
    return password
        
#password = attack_key_sbit()


In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

Capturez un grand nombre de traces (typiquement 5000) en utilisant la même méthode que dans la partie précédente de ce TP, puis écrivez un code python qui effectue un DPA en considérant le bit `i` (commencez avec `i = 0` puis vous modifierez votre code pour qu'il fonctionne pour n'importe quel bit après) sur le premier octet de la clé.

Étendez ce code pour attaquer tous les octets de clé. 
Étendez ce code pour qu'il fonctionne pour n'importe quel bit en sortie de la SBox.

Rédigez des commentaires pour expliquer votre code (et l'attaque que vous avez mise en oeuvre).

**Aide** 
* Vous pouvez utiliser la fonction `numpy.save(filename, array)` afin de sauvegarder vos traces dans un fichier ; cela évite d'avoir à capturer des traces à chaque fois que vous voulez travailler sur le code de l'attaque.
* Pour récupérer les données sauvegardées, vous devez utiliser `array = numpy.load(filename)`
* Si l'attaque ne fonctionne pas, cela peut aussi venir du fait que vous n'avez pas choisi le bon `offset` et `samples` : l'attaque peut facilement récupérer d'autres parties de l'opération AES (vous pouvez essayer avec différentes valeurs pour voir ce point, vous devriez en fait utiliser les valeurs trouvées -- mieux encore ne regarder que les échantillons dans un petit intervalle autour de chaque valeur trouvée -- dans la première partie de ce TME lors de l'analyse des traces de puissance en considérant le HW de la sortie de la Sbox)
* Si l'attaque ne fonctionne pas, vous pouvez également essayer d'augmenter le nombre de traces que vous capturez. 
* Cependant, en fonction de votre carte, il se peut que vous ne puissiez pas casser tous les octets clés, faites-nous savoir si vous rencontrez un tel cas (avant, testez votre code en considérant un bit différent, puis testez-le avec une autre carte pour voir si la carte est la vraie racine du problème !)

**Analyse de l'efficacité de l'attaque**
* Essayez votre DPA en considérant différents bits, et analysez son efficacité en fonction du bit. 

* Vous pouvez également essayer de trouver le nombre minimum de traces nécessaires pour réussir à récupérer tous les octets clés (ou ceux que vous êtes en mesure de récupérer).

## Attaque DPA Multi-bit

En utilisant un grand nombre de traces (typiquement le même que celui utilisé pour la DPA mono-bit), écrivez un code python qui effectue une DPA multi-bit sur le premier octet de la clé avant d'étendre ce code pour attaquer tous les octets de la clé. Vous écrirez quelques commentaires pour expliquer votre code (et l'attaque que vous avez mise en oeuvre). 

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

trace_array = np.load("trace_array.npy")
textin_array = np.load("textin_array.npy")
known_keys = np.load("known_keys.npy")


# critical points for bytes {0 .. 15}
hw_leak = [49, 269, 488, 708, 96, 316, 536, 756, 144, 364, 584, 804, 220, 440, 660, 836]

# range to consider around the critical point 
alpha = 2

N = 100 # max = 5000

def attack_byte(bnum) :
    M = 0
    best_key = 0
    
    for key_guess in range(256) :
        # sort traces depending on their supposed hamming weights
        # hw < 4
        hw_low = []
        # hw >= 4
        hw_high = []
        
        for tnum in range(0, N): 
            textin = textin_array[tnum][bnum]
            # compute the supposed sbox out considering the key byte guess
            sbox_out = intermediate(textin, key_guess)
            # compute the hamming weight of the sbox out 
            h = HW[sbox_out]
            # add the corresponding trace to the right list 
            if h < 4 : 
                hw_low.append(tnum)
            else : 
                hw_high.append(tnum)
        # small range around the hamming weight leak 
        ra = range(hw_leak[bnum] - alpha, hw_leak[bnum] + alpha + 1)
        
        mean_trace_hw_low = [0] * len(ra)
        mean_trace_hw_high = [0] * len(ra)
        
        # compute the mean values of the traces belonging to the two groups 
        # the result is 1 (mean) trace
        for j in range(len(ra)) :
            mean_trace_hw_low[j] = np.mean([trace_array[tnum][ra[0]+j] for tnum in hw_low])
            mean_trace_hw_high[j] = np.mean([trace_array[tnum][ra[0]+j] for tnum in hw_high])
        # compute the difference between the mean values
        diff = np.array(mean_trace_hw_low) - np.array(mean_trace_hw_high)        
        sabs = np.sum(np.abs(diff))
        
        # store the byte that has the largest mean value difference s
        if sabs > M :
            M = sabs
            best_key = key_guess
    return best_key
    
def attack_key_mbit() :
    password = ""
    for bnum in range(16) :
        by = attack_byte(bnum)
        password += "{:02x}".format(by)
    #print("found password = \t", password)
    #print("actual password = \t", key)
    return password

#m = 256
#for i in range(0, m, 4) :
    #N = i
    #password = attack_key_multibit()
    #print("traces = ", m)
    #print(compare_keys(key, password))
    #print(error_count(key, password))

#print(compare_keys(key, password))
#print(error_count(key, password))        

In [ ]:
from time import time

trace_array = np.load("trace_array.npy")
textin_array = np.load("textin_array.npy")
known_keys = np.load("known_keys.npy")
error_sbit = []
error_mbit = []
time_sbit = []
time_mbit = []
r = range(0, 512+1, 16)
print(list(r))
for i in r:
    N = i
    print(N)
    key = ""
    #print(known_keys[0])
    for i in range(0, 16) :
        by = known_keys[0][i]
        key += "{:02x}".format(by)
    #print(key)
    
    start = time()
    guessed_key_sbit = attack_key_sbit()
    end = time()
    time_sbit.append(end-start)
    
    start = time()
    guessed_key_mbit = attack_key_mbit()
    end = time()
    time_mbit.append(end-start)
    #print(guessed_key_sbit)
    #print(guessed_key_mbit)
    #print(error_count(key, guessed_key_sbit))
    #print(error_count(key, guessed_key_mbit))
    error_sbit.append(error_count(key, guessed_key_sbit))
    error_mbit.append(error_count(key, guessed_key_mbit))
    print(error_sbit)
    print(error_mbit)
    print(time_sbit)
    print(time_mbit)
print(list(r))
print(error_sbit)
print(error_mbit)
print(time_sbit)
print(time_mbit)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d

output_notebook()

xrange = [16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416, 432, 448, 464, 480, 496, 512, 528]
sbit = [15, 16, 9, 8, 6, 7, 4, 3, 5, 3, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
mbit = [12, 3, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

time_sbit = [5.225416421890259, 7.44797682762146, 9.735242366790771, 12.013313293457031, 14.347801208496094, 16.15178871154785, 18.45497417449951, 20.68447756767273, 22.907174348831177, 25.37883186340332, 27.768035411834717, 29.76925539970398, 32.03330206871033, 34.40781307220459, 36.811065912246704, 39.35733675956726, 40.93652367591858, 43.14633631706238, 45.45450305938721, 47.58788704872131, 50.09656763076782, 52.26060080528259, 54.185081243515015, 56.90841770172119, 57.936240434646606, 60.7835259437561, 62.1680121421814, 64.52762961387634, 66.96742296218872, 69.08459091186523, 71.20212864875793, 73.76131987571716, 80.60242915153503]
time_mbit = [0.6429474353790283, 0.9338877201080322, 1.1837716102600098, 1.5125494003295898, 1.7068195343017578, 1.9920308589935303, 2.269242286682129, 2.5459787845611572, 2.8247671127319336, 3.120332717895508, 3.4026939868927, 3.6537222862243652, 3.919450044631958, 4.223193168640137, 4.511546850204468, 4.773949384689331, 5.0220582485198975, 5.294931888580322, 5.5920610427856445, 5.845255374908447, 6.153587818145752, 6.39685845375061, 6.644589424133301, 7.057284832000732, 7.382230281829834, 7.5452961921691895, 7.71592903137207, 8.058278560638428, 8.246114730834961, 8.657761573791504, 8.61436128616333, 8.986669778823853, 9.567970991134644]

p = figure(title="Wrong bytes vs number of traces")
p.xaxis.axis_label = "Number of traces analyzed"
p.yaxis.axis_label = "Number of wrong bytes in recovered key"

p.line(xrange, sbit, line_color="red", legend_label='sbit errors', line_width=2)
p.line(xrange, mbit, line_color='orange', legend_label='mbit errors', line_width=2)
p.y_range = Range1d(start = -1, end = 17)
p.extra_y_ranges = {"y2": Range1d(start = 0, end = 82)}
p.add_layout(LinearAxis(y_range_name = "y2", axis_label="Execution time (in s)"), 'right')
p.line(xrange, time_sbit, color = "blue", y_range_name = "y2", legend_label='sbit time')
p.line(xrange, time_mbit, color = "turquoise", y_range_name = "y2", legend_label='mbit time')


show(p)

In [ ]:
nkeys = 16
r = range(16, 256+1, 16)
print(list(r))
res = np.zeros((16, 16), dtype=int, order='C')
j = 0
print(res)
for i in r:
    N = i
    print(N)
    for key_number in range(nkeys) :
        a = np.load("k"+str(key_number)+"_trace_array.npy")
        trace_array = a
        b = np.load("k"+str(key_number)+"_textin_array.npy")
        textin_array = b
        c = np.load("k"+str(key_number)+"_known_keys.npy")
        known_keys = c
        key = ""
        #print(known_keys[0])
        for i in range(0, 16) :
            by = known_keys[0][i]
            key += "{:02x}".format(by)
        #print(key)

        guessed_key_mbit = attack_key_mbit()

        #print(guessed_key_sbit)
        #print(guessed_key_mbit)
        #print(error_count(key, guessed_key_sbit))
        #print(error_count(key, guessed_key_mbit))
        ec = compare_keys(key, guessed_key_mbit)
        for bidx in range(len(ec)):
            if ec[bidx] == False :
                res[bidx][j]+=1
    j+=1
    print(res)
print(list(r))
#print(error_sbit)
#print(error_mbit)
#print(time_sbit)
#print(time_mbit)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d
from bokeh.palettes import inferno, viridis

color_mapper = (inferno(16))
#color_mapper = (viridis(16))
print(color_mapper)

res_plot =   [[12,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [10,  3,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [13,  5,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [13,  6,  3,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [14,  5,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [13,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [15,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [16,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [13,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [10,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [14,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [16,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [10,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [14,  4,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
 [16, 14, 11, 12,  6,  5,  5,  5,  4,  3,  1,  1,  0,  0,  0,  1]]
#print(res)
for i in range(16) :
    for j in range(16) :
        res_plot[i][j] = ((16 - res_plot[i][j]) / 16.) * 100

#print(np.array(res_plot))
output_notebook()

xrange = [16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256]
p = figure(title="Byte recovery statistics (testing on 16 different sets of sampls)")
p.xaxis.axis_label = "Number of traces analyzed"
p.yaxis.axis_label = "Byte recovery percentage"

for i in range(8) :
    p.line(xrange, res_plot[i], line_color=color_mapper[15-i], legend_label='b'+str(i))

show(p)

**Analyse de l'efficacité des attaques**
* Comparez l'efficacité de la DPA multi-bit avec celle de la DPA single-bit en considérant le même nombre de traces.
* Comparez le nombre de traces nécessaires pour récupérer tous les octets de clé (ou tous les octets de clé que vous êtes en mesure de récupérer sur votre carte).

## Conclusion

Vous avez (normalement) cassé l'AES en utilisant une attaque DPA ! Comme vous l'avez peut-être découvert au cours de ce TME, la méthode de la différence de moyennes pour casser la clé de l'AES peut poser un certain nombre de problèmes :

* Elle est assez sensible au bruit
* L'attaque nécessite généralement beaucoup de traces. Les implémentations AES logicielles non protégées sont assez faibles contre l'analyse de puissance, mais elles ont tout de même nécessité des milliers de traces pour être cassées

Néanmoins, l'utilisation d'une attaque par différence de moyennes peut s'avérer très utile dans plusieurs contextes.

## Rapport ## 

Vous devez rédiger un rapport dans lequel vous devez expliquer les différentes étapes de ce TME et du TME 3, ainsi que vos conclusions.

Vous devez expliquer la première partie sur l'identification des POI (ses objectifs et ce que vous avez appris) et donner les points que vous avez identifiés pour chaque octet clé, comment vous avez sélectionné un point parmi plusieurs le cas échéant.

Vous devez expliquer comment vous avez mis en œuvre les attaques DPA, vous devez donner et commenter vos codes.

Vous devez également fournir et discuter certains résultats : il est recommandé de calculer et de fournir des statistiques sur le succès de l'attaque DPA en fonction de l'octet sélectionné (ou du bit dans un octet) et du nombre de traces.

Vous devez également comparer l'efficacité des attaques DPA mono et multi-bits. N'oubliez pas non plus de tester votre attaque avec différentes clés, vous pouvez les changer (manuellement ou non) !